# Hyperparameters HMM

Launch `hyperparameters.sh`!

* Fraction scores used for HMM signal, noise and ratio estimation, guards decent SNR in model<br>
`SPAN_DEFAULT_HMM_ESTIMATE_SNR = 0.1`

* Minimal low state mean threshold, guards against too broad peaks<br>
`SPAN_DEFAULT_HMM_LOW_THRESHOLD = 0.3`

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import pandas as pd
import seaborn as sns
from tqdm.auto import tqdm

sns.set_style("whitegrid")
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
import tempfile

def bedl(file):
    try:
        tf = pd.read_csv(file, sep='\t', header=None)
        return tf[2] - tf[1]
    except:
        return np.zeros(0)  # Empty file


def lines(file):
    try:
        tf = pd.read_csv(file, sep='\t', header=None)
        return len(tf)
    except:
        return 0  # Empty file

def sorted_file(file):
    ts = tempfile.mktemp()
    ! cat {file} | sort -k1,1 -k2,2n -k3,3n > {ts}
    return ts

def d(a, b):
    return a / b if b != 0 else 0.0

In [ ]:
SNRS = [0, 0.001, 0.01, 0.05, 0.1, 0.3]
LOWS = [0, 0.01, 0.1, 0.2, 0.3, 0.5]

PATH = os.path.expanduser('~/data/2025_hyperparameters')
rows = []
for (path, f) in tqdm([(PATH, f) for f in os.listdir(PATH)]):
    if not f.endswith('.peak'):
        continue
    if f.startswith('HSMM_H3K36me3_rep1_'):
        mod, cell, rep = 'H3K36me3', 'HSMM', 'rep1'
    elif f.startswith('HSMM_H3K36me3_rep2_'):
        mod, cell, rep = 'H3K36me3', 'HSMM', 'rep2'
    elif f.startswith('GM12878_H3K4me3_rep1_'):
        mod, cell, rep = 'H3K4me3', 'GM12878', 'rep1'
    elif f.startswith('GM12878_H3K4me3_rep2_'):
        mod, cell, rep = 'H3K4me3', 'GM12878', 'rep2'
    elif f.startswith('GM12878_H3K36me3_rep1_'):
        mod, cell, rep = 'H3K36me3', 'GM12878',' rep1'
    elif f.startswith('GM12878_H3K36me3_rep2_'):
        mod, cell, rep = 'H3K36me3', 'GM12878', 'rep2'
    elif f.startswith('HepG2_H3K27ac_rep1_'):
        mod, cell, rep = 'H3K27ac', 'HepG2', 'rep1'
    elif f.startswith('HepG2_H3K27ac_rep2_'):
        mod, cell, rep = 'H3K27ac', 'HepG2', 'rep2'
    elif f.startswith('CD34_H3K27ac_hg38_ENCFF026YFQ_'):
        mod, cell, rep = 'H3K27ac', 'CD34', ''
    elif f.startswith('OD1_k27ac_hg19_'):
        mod, cell, rep = 'H3K27ac', 'YO', 'OD9'
    elif f.startswith('YD11_k27ac_hg19_'):
        mod, cell, rep = 'H3K27ac', 'YO', 'YD11'
    else:
        continue
    snr, low = None, None
    for s in SNRS:
        for l in LOWS:
            if f'_{s}_{l}.peak' in f:
                snr, low = s, l
                break
    if snr is None or low is None:
        continue
    peaks_path = os.path.join(path, f)
    peaks = lines(peaks_path)
    lens = list(sorted(bedl(peaks_path)))
    lens = lens[int(len(lens) * 0.1): int(len(lens) * 0.9)]
    avl = np.sum(lens) / len(lens)
    rows.append((peaks_path, mod, cell, rep, snr, low, peaks, avl))

t = pd.DataFrame(rows, columns=['file', 'modification', 'cell', 'replicate', 'snr', 'low', 'peaks', 'length'])
t['name'] = t['cell'] + '_' + t['replicate']
t['constraint'] = 'snr_' + t['snr'].astype(str) + '_low_' + t['low'].astype(str)

# Visual aesthetics
t['peaks'] += np.random.normal(0, 10.0, len(t))
t['length'] += np.random.normal(0, 20.0, len(t))
t.loc[t['length'] < 0, 'length'] = 0
t.sort_values(by=['modification', 'name', 'snr', 'low'], inplace=True)
t.sample(3)

In [ ]:
for m, tm in t.groupby('modification'):
    plt.figure(figsize=(6, 3))
    ax = plt.axes()
    sns.lineplot(data=tm, x="low", y='peaks', hue='snr', style='name', palette='tab10', alpha=0.5, ax=ax)
    sns.scatterplot(data=tm[(tm['low'] == 0) & (tm['snr'] == 0)], x="low", y='peaks', hue='snr',
                 palette='tab10', ax=ax, legend=False)
    # Put a legend to the right of the current axis
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_title(f'{m} peaks')
    ax.set_ylim(bottom=0)
    plt.tight_layout()
    plt.show()



In [ ]:
plt.figure(figsize=(6, 3))
ax = plt.axes()
sns.boxplot(data=t, x="snr", y='peaks', hue='modification', ax=ax)
# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_title(f'Peaks')
plt.tight_layout()
plt.show()


In [ ]:
for m, tm in t.groupby('modification'):
    plt.figure(figsize=(6, 3))
    ax = plt.axes()
    sns.lineplot(data=tm, x="low", y='length', hue='snr', style='name', palette='tab10', alpha=0.5, ax=ax)
    sns.scatterplot(data=tm[(tm['low'] == 0) & (tm['snr'] == 0)], x="low", y='length', hue='snr',
                    palette='tab10', ax=ax, legend=False)

    # Put a legend to the right of the current axis
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_title(f'{m} average length')
    ax.set_ylim(bottom=0)
    plt.tight_layout()
    plt.show()



In [ ]:
plt.figure(figsize=(6, 3))
ax = plt.axes()
sns.boxplot(data=t, x="snr", y='length', hue='modification', ax=ax)
# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_title(f'Average length')
plt.tight_layout()
plt.show()


In [ ]:
import pyranges as pr

reps_overlap = pd.DataFrame(columns=['modification', 'cell', 'snr', 'low',
                                     'peaks1', 'peaks1_len', 'peaks2', 'peaks2_len',
                                     'peaks1_overlap', 'peaks2_overlap', 'peaks_overlap_len'], dtype=object)

for (m, c, snr, low), tm in tqdm(t.groupby(['modification', 'cell', 'snr', 'low'])):
    if len(tm) != 2:
        continue
    file1, file2 = pr.read_bed(sorted_file(tm['file'].values[0])), pr.read_bed(sorted_file(tm['file'].values[1]))
    peaks1, peaks2 = len(file1), len(file2)
    peaks1_len, peaks2_len = file1.lengths().sum(), file2.lengths().sum()
    if peaks1 <= 0 or peaks2 <= 0:
            reps_overlap.loc[len(reps_overlap)] = (m, c, snr, low, peaks1, peaks1_len, peaks2, peaks2_len, 0, 0, 0)
    else:
        overlap1 = len(file1.overlap(file2))
        overlap2 = len(file2.overlap(file1))
        overlap_len = file1.intersect(file2).lengths().sum()
        reps_overlap.loc[len(reps_overlap)] = (m, c, snr, low, peaks1, peaks1_len, peaks2, peaks2_len, overlap1, overlap2, overlap_len)
reps_overlap['jaccard'] = [
    d(lo, l1 + l2 - lo)
    for l1, l2, lo in zip(reps_overlap['peaks1_len'], reps_overlap['peaks2_len'], reps_overlap['peaks_overlap_len'])
]
reps_overlap

In [ ]:
for m, tm in reps_overlap.groupby('modification'):
    plt.figure(figsize=(6, 3))
    ax = plt.axes()
    sns.lineplot(data=tm, x="low", y='jaccard', hue='snr', style='cell', palette='tab10', alpha=0.5, ax=ax)
    sns.scatterplot(data=tm[(tm['low'] == 0) & (tm['snr'] == 0)], x="low", y='jaccard', hue='snr',
                    palette='tab10', ax=ax, legend=False)

    # Put a legend to the right of the current axis
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_title(f'{m} Jaccard')
    ax.set_ylim(bottom=0)
    plt.tight_layout()
    plt.show()


In [ ]:
plt.figure(figsize=(6, 3))
ax = plt.axes()
sns.boxplot(data=reps_overlap, x="snr", y='jaccard', hue='modification', ax=ax)
# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_title('Jaccard')
plt.tight_layout()
plt.show()